In [1]:
%matplotlib notebook
import numpy as np
import itertools

from config import CFG
from dataset import load_and_prepare_data
from kde_classes import Model, KDE

from models.multi_gaussian import settings, grid

import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import axes3d

Welcome to JupyROOT 6.14/04


In [2]:
mg = np.load(CFG['paths']['mg_mc'])
model = Model(mg, settings)

In [3]:
kde = KDE(model)

Using ones as weight.
                   y INFO: Creating 1D phase space in range (-5.000000, 5.000000)
                   x INFO: Creating 1D phase space in range (-5.000000, 5.000000)
        PhspCombined INFO: Creating combined phase space of 2 components
        PhspCombined INFO: Adding component phase space "y", dim=1
        PhspCombined INFO: Adding component phase space "x", dim=1
        PhspCombined INFO: Resulting dimensionality is 2


In [ ]:
binned_kernel_density = kde.generate_binned_kernel_density()

In [ ]:
out_bins = []
for key in settings:
    out_bins.append(np.linspace(settings[key]['range'][0],
                                settings[key]['range'][1],
                                settings[key]['nbins']))
coords = np.array(list(itertools.product(*out_bins)))

In [ ]:
nbins = 100

print('Evaluate KDEs:')
pdf_vals = np.asarray([kde.eval_point(coord) for coord in coords])
shape = np.ones(len(settings.keys()), dtype=int)*nbins
pdf_vals = pdf_vals.reshape(*shape)

In [ ]:
X, Y = np.meshgrid(*out_bins)

In [ ]:
plt.figure()

plt.pcolormesh(X, Y, pdf_vals)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, pdf_vals)
ax.set_xlabel('x, $\sigma=1$')
ax.set_ylabel('y, $\sigma=2$')

Generate adaptive kde

In [ ]:
adaptive_kernel_density = kde.generate_adaptive_kernel_density()

In [ ]:
nbins = 100

print('Evaluate KDEs:')
adaptive_pdf_vals = np.asarray([kde.eval_point(coord) for coord in coords])
shape = np.ones(len(settings.keys()), dtype=int)*nbins
adaptive_pdf_vals = adaptive_pdf_vals.reshape(*shape)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, adaptive_pdf_vals)
ax.set_xlabel('x, $\sigma=1$')
ax.set_ylabel('y, $\sigma=2$')

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
h = ax.hist2d(mg['x'], mg['y'], bins=[100, 100], range=[[-5, 5], [-5, 5]])
fig.colorbar(h[3], ax=ax)


In [ ]:
llh, zeros = kde.cross_validate()

In [5]:
result = kde.cross_validate_bandwidths()

[[0.3, 0.4, 0.5, 0.6], array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])]
(0.3, 0.1)
Using ones as weight.
Using ones as weight.
Using ones as weight.
Using ones as weight.
Using ones as weight.
('llh, zeros:', [-6806.20104066579, -6854.918897898333, -6861.111261419833, -6772.452717420845, -6730.284755443707], [0, 0, 0, 0, 0])
(0.3, 0.2)
Using ones as weight.
Using ones as weight.
Using ones as weight.
Using ones as weight.
Using ones as weight.
('llh, zeros:', [-6815.157360544591, -6793.770428270223, -6825.943632302338, -6816.141835571063, -6730.921110808593], [0, 0, 0, 0, 0])
(0.3, 0.30000000000000004)
Using ones as weight.
Using ones as weight.
Using ones as weight.
Using ones as weight.
Using ones as weight.
('llh, zeros:', [-6766.899004208487, -6792.915799592356, -6802.706914481705, -6816.548684837609, -6730.85644264079], [0, 0, 0, 0, 0])
(0.3, 0.4)
Using ones as weight.
Using ones as weight.
Using ones as weight.
Using ones as weight.
Using ones as weight.
('llh, zeros:',

Using ones as weight.
('llh, zeros:', [-6764.761087810304, -6783.886995898462, -6795.594045830292, -6781.255134199559, -6734.616876637784], [0, 0, 0, 0, 0])
(0.6, 0.30000000000000004)
Using ones as weight.
Using ones as weight.
Using ones as weight.
Using ones as weight.
Using ones as weight.
('llh, zeros:', [-6777.737522395422, -6782.549948666715, -6785.306940522434, -6768.417527709938, -6732.467448421712], [0, 0, 0, 0, 0])
(0.6, 0.4)
Using ones as weight.
Using ones as weight.
Using ones as weight.
Using ones as weight.
Using ones as weight.
('llh, zeros:', [-6809.8370501164645, -6776.526430785484, -6807.122503449411, -6761.357283141755, -6724.936220218769], [0, 0, 0, 0, 0])
(0.6, 0.5)
Using ones as weight.
Using ones as weight.
Using ones as weight.
Using ones as weight.
Using ones as weight.
('llh, zeros:', [-6800.797090134229, -6774.825591601823, -6797.897367658471, -6758.777173525109, -6723.12021169179], [0, 0, 0, 0, 0])
(0.6, 0.6)
Using ones as weight.
Using ones as weight.
Usin

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

